In [1]:
from bs4 import BeautifulSoup
import json
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from pymystem3 import Mystem
import pickle
import re
from nltk.corpus import stopwords
import nltk
import sqlite3

In [2]:
conn = sqlite3.connect(r"../../diplom.db")
cur = conn.cursor()

In [5]:
i, date, text, count = cur.execute("""SELECT ID, DDATE, TRANSCRIPT, DEMOCRACY_COUNT FROM PRESIDENT WHERE ID = 1773761;""").fetchone()

In [6]:
with open(f'{i}.txt', 'w', encoding='utf-8') as f:
       f.write(text)

In [7]:
cur.close()
conn.close()

In [16]:
res = cur.execute("""SELECT ID, DDATE, TRANSCRIPT, DEMOCRACY_COUNT FROM PRESIDENT ORDER BY DEMOCRACY_COUNT DESC LIMIT 100;""")

In [17]:
for i, date, text, count in res:
    print(i, date, count)
    with open(f'{i}.txt', 'w', encoding='utf-8') as f:
        f.write(text)

1770470 2010-09-10 00:00:00 99
1770473 2010-09-10 00:00:00 56
1773407 2005-02-24 00:00:00 28
1775141 2007-09-14 00:00:00 28
1768664 2010-01-22 00:00:00 23
1772015 2002-01-15 00:00:00 19
1779179 2012-07-19 00:00:00 19
1776254 2014-10-24 00:00:00 18
1769256 2012-11-12 00:00:00 17
1773528 2005-05-09 00:00:00 17
1771161 2013-09-19 00:00:00 16
1770144 2010-04-26 00:00:00 15
1773403 2005-02-22 00:00:00 15
1773499 2005-04-25 00:00:00 15
1776899 2015-10-22 00:00:00 14
1775853 2009-04-13 00:00:00 13
1778124 2009-09-20 00:00:00 13
1778290 2017-10-19 00:00:00 13
1770711 2010-12-24 00:00:00 12
1774253 2006-07-12 00:00:00 12
1770223 2010-05-28 00:00:00 11
1770226 2010-05-28 00:00:00 11
1772719 2003-10-04 00:00:00 11
1777343 2016-06-17 00:00:00 11
1777858 2009-09-10 00:00:00 11
1780292 2008-07-03 00:00:00 11
1774583 2007-02-10 00:00:00 10
1777281 2012-04-26 00:00:00 10
1773761 2005-09-16 00:00:00 9
1774950 2007-06-21 00:00:00 9
1777047 2016-01-12 00:00:00 9
1777721 2016-12-23 00:00:00 9
1770824 2013

In [18]:
conn.close()